In [1]:
from genetic import Job, ga_run
import pandas as pd

Import success


In [2]:
df = pd.read_csv("resources/new_example.csv")
#df = pd.read_csv("resources/example_problems(p2).csv")

df.head

<bound method NDFrame.head of     a  3  10
0   b  2   5
1   c  5  10
2   d  1  10
3   e  3  30
4   f  1  20
5   g  2  10
6   h  4  25
7   i  2  15
8   j  3  35
9   k  1   5
10  l  4  40
11  m  2  20
12  n  3  15
13  o  5  50
14  p  1  30
15  q  2  12
16  r  4  22
17  s  3  18
18  t  1   8
19  u  5  45
20  v  2  28
21  w  3  33
22  x  4  12
23  y  1   7
24  z  2   9>

In [3]:
col1 = df.iloc[:,0]
print(col1)

0     b
1     c
2     d
3     e
4     f
5     g
6     h
7     i
8     j
9     k
10    l
11    m
12    n
13    o
14    p
15    q
16    r
17    s
18    t
19    u
20    v
21    w
22    x
23    y
24    z
Name: a, dtype: object


In [4]:
id_ = df.iloc[:,0]
deadline = df.iloc[:,1]
profit = df.iloc[:,2]

jobs = [Job(id_[i], deadline[i], profit[i]) for i in range(len(id_))]
max_profit, job_count, best_schedule = ga_run(
    jobs, 
    pop_size=100,
    generations=500, 
    mutation_rate=0.35,
    tournament_size=10,
    selection_method="roulette",
    crossover_method="pmx",
    mutation_method="random_reset")

print(f"\nNumber of jobs completed: {job_count}, Maximum profit: {max_profit}")

Initial Population of Chromosomes:
Chromosome 1: Schedule = [5, 0, 11, 9, 12, 22, 13, 24, 16, 23, 7, 17, 18, 19, 1, 10, 15, 21, 4, 2, 8, 14, 6, 3, 20]
Chromosome 2: Schedule = [22, 10, 14, 1, 18, 13, 0, 11, 16, 23, 6, 17, 9, 7, 8, 4, 15, 5, 12, 19, 24, 3, 21, 2, 20]
Chromosome 3: Schedule = [15, 12, 14, 24, 2, 13, 6, 23, 16, 1, 21, 7, 3, 5, 0, 4, 9, 8, 17, 18, 20, 10, 19, 11, 22]
Chromosome 4: Schedule = [24, 22, 5, 10, 20, 12, 0, 3, 16, 18, 8, 1, 6, 11, 9, 14, 13, 2, 19, 23, 7, 4, 17, 21, 15]
Chromosome 5: Schedule = [2, 9, 16, 4, 1, 15, 6, 14, 23, 10, 12, 24, 5, 22, 7, 8, 20, 13, 11, 17, 3, 19, 18, 21, 0]
Chromosome 6: Schedule = [20, 15, 23, 2, 17, 0, 13, 14, 4, 12, 6, 3, 19, 16, 5, 22, 24, 18, 21, 1, 9, 8, 7, 10, 11]
Chromosome 7: Schedule = [8, 7, 17, 21, 1, 19, 6, 16, 12, 3, 18, 11, 10, 5, 22, 14, 24, 15, 13, 0, 2, 23, 20, 9, 4]
Chromosome 8: Schedule = [18, 16, 5, 15, 8, 7, 22, 20, 23, 17, 6, 0, 21, 11, 2, 14, 13, 12, 24, 3, 19, 10, 4, 1, 9]
Chromosome 9: Schedule = [13, 14, 9, 

In [5]:
print(f"\nNumber of jobs completed: {job_count}, Maximum profit: {max_profit}")
print(f"Genetic Algorithm Scheduled Jobs:\n {best_schedule}")


Number of jobs completed: 5, Maximum profit: 190
Genetic Algorithm Scheduled Jobs:
 ['o', 'l', 'e', 'h', 'u']


### Greedy Comparison
To judge the maximum profit output for the genetic algorithm, it will be compared to the output of a greedy algorithm. See the implementation of the algorithm below:

In [6]:
def greedy_job_scheduling(jobs):
    jobs.sort(key=lambda x: x.profit, reverse=True)
    max_deadline = max(job.deadline for job in jobs)
    time_slots = [-1] * max_deadline
    total_profit = 0
    completed_jobs = 0

    for job in jobs:
        for t in range(job.deadline - 1, -1, -1):
            if time_slots[t] == -1:
                time_slots[t] = job.id
                total_profit += job.profit
                completed_jobs += 1
                break

    print("Greedy Algorithm Scheduled Jobs:\n", time_slots)
    return completed_jobs, total_profit

In [7]:
greedy_jobs, greedy_profit = greedy_job_scheduling(jobs)
print(f"\nGreedy Algorithm:\n Jobs completed: {greedy_jobs}. Maximum profit: {greedy_profit}.")

Greedy Algorithm Scheduled Jobs:
 ['w', 'j', 'l', 'u', 'o']

Greedy Algorithm:
 Jobs completed: 5. Maximum profit: 203.


In the output above the optimal sequence of jobs with the maximum possible profit.